# 2020.03.28 (Sat)

## Base model: spaCy

In [1]:
#!prodigy train ner cord_19_abstracts_match models/en_core_web_lg_no_ner --output models/2020_03_28_match/en_rf_web_lg

✔ Loaded model 'models/en_core_web_lg_no_ner'
Created and merged data for 86 total examples
Using 43 train / 43 eval (split 50%)
Component: ner | Batch size: compounding | Dropout: 0.2 | Iterations: 10
ℹ Baseline accuracy: 0.000

=========================== ✨  Training the model ===========================

#    Loss       Precision   Recall     F-Score 
--   --------   ---------   --------   --------
 1    2260.23       3.822      5.714      4.580                                 
 2     304.20      96.203     72.381     82.609                                 
 3      35.94      93.069     89.524     91.262                                 
 4       7.71      91.262     89.524     90.385                                 
 5       6.89      94.000     89.524     91.707                                 
 6       6.26      94.949     89.524     92.157                                 
 7       4.15      94.000     89.524     91.707                                 
 8      10.58      92.157   

## Base model: scispaCy

In [2]:
#!prodigy train ner cord_19_abstracts_match models/en_core_sci_lg_no_ner --output models/2020_03_28_match/en_rf_sci_lg

✔ Loaded model 'models/en_core_sci_lg_no_ner'
Created and merged data for 86 total examples
Using 43 train / 43 eval (split 50%)
Component: ner | Batch size: compounding | Dropout: 0.2 | Iterations: 10
ℹ Baseline accuracy: 0.000

=========================== ✨  Training the model ===========================

#    Loss       Precision   Recall     F-Score 
--   --------   ---------   --------   --------
 1    1864.33       3.333      2.941      3.125                                 
 2     303.79      93.902     75.490     83.696                                 
 3      22.39      92.784     88.235     90.452                                 
 4      17.47      94.792     89.216     91.919                                 
 5       5.36      93.814     89.216     91.457                                 
 6       6.78      92.857     89.216     91.000                                 
 7       6.09      93.814     89.216     91.457                                 
 8       4.87      94.792   

# 2020.03.29 (Sun)

## Test the model trained on Sat

In [1]:
import spacy
import warnings
from pprint import pprint

def test_model(model_path):
    warnings.filterwarnings("ignore",category=UserWarning)
    
    nlp = spacy.load(model_path)
    texts = [
    "Known risk factors for the disease are: older age, male gender, diabetes and leukemia. Female patiens and children are less susceptible.",
    "Diabetes is a known risk factor.",
    "Leukemia is a risk factor, too."
    ]

    for text in texts:
        doc = nlp(text)

        spacy.displacy.render(doc, style='ent', jupyter=True)
        pprint([(ent.text, ent.label_) for ent in doc.ents])

In [2]:
test_model("models/2020_03_28_match/en_rf_web_lg")

[('age', 'RISK_FACTOR'), ('male', 'RISK_FACTOR'), ('diabetes', 'RISK_FACTOR')]


[('Diabetes', 'RISK_FACTOR')]


[]


Only the factors that appeared during the training are detected. _Leukemia_ is **not highlighed** and that means that the NER model did not learn to recognize risk factors based on the sentence structure.

## Train new models after the `teach` session

In [1]:
#!prodigy train ner cord_19_abstracts_teach models/en_core_web_lg_no_ner --output models/2020_03_29_teach/en_rf_web_lg

✔ Loaded model 'models/en_core_web_lg_no_ner'
Created and merged data for 393 total examples
Using 197 train / 196 eval (split 50%)
Component: ner | Batch size: compounding | Dropout: 0.2 | Iterations: 10
ℹ Baseline accuracy: 0.000

=========================== ✨  Training the model ===========================

#    Loss       Precision   Recall     F-Score 
--   --------   ---------   --------   --------
 1     333.36      89.669     92.735     91.176                                 
 2      64.66      92.982     90.598     91.775                                 
 3      53.47      94.492     95.299     94.894                                 
 4      45.26      94.492     95.299     94.894                                 
 5      29.42      94.958     96.581     95.763                                 
 6      30.44      96.170     96.581     96.375                                 
 7      23.82      95.745     96.154     95.949                                 
 8       5.78      95.319

In [2]:
#!prodigy train ner cord_19_abstracts_teach models/en_core_sci_lg_no_ner --output models/2020_03_29_teach/en_rf_sci_lg

✔ Loaded model 'models/en_core_sci_lg_no_ner'
Created and merged data for 393 total examples
Using 197 train / 196 eval (split 50%)
Component: ner | Batch size: compounding | Dropout: 0.2 | Iterations: 10
ℹ Baseline accuracy: 0.000

=========================== ✨  Training the model ===========================

#    Loss       Precision   Recall     F-Score 
--   --------   ---------   --------   --------
 1     289.28      90.991     91.818     91.403                                 
 2      54.12      92.237     91.818     92.027                                 
 3      84.39      91.593     94.091     92.825                                 
 4      46.45      93.213     93.636     93.424                                 
 5      19.43      95.000     95.000     95.000                                 
 6      15.25      93.665     94.091     93.878                                 
 7       4.07      93.363     95.909     94.619                                 
 8      11.23      93.722

In [3]:
test_model("models/2020_03_29_teach/en_rf_web_lg")

[('age', 'RISK_FACTOR'),
 ('male gender', 'RISK_FACTOR'),
 ('diabetes', 'RISK_FACTOR')]


[('Diabetes', 'RISK_FACTOR')]


[]


Better model scores, but _leukemia_ still **not detected**.

# 2020.03.30 (Mon)
## Starting the annotations from scratch, in strict mode
Terms like _male_ or _age_ will only be marked, when they are mentioned in the context of COVID-19 risk factors, i.e. not in articles about animals or children.

In [5]:
!prodigy ner.manual cord_19_abstracts_strict models/en_core_sci_lg_no_ner data/raw/cord_19_abstracts_filtered.jsonl \
  --label RISK_FACTOR --patterns patterns/RF_list_2020.03.17.20037572.jsonl

Using 1 label(s): RISK_FACTOR
Added dataset cord_19_abstracts_strict to database SQLite.

✨  Starting the web server at http://localhost:8080 ...
Open the app in your browser and start annotating!

^C

✔ Saved 41 annotations to database SQLite
Dataset: cord_19_abstracts_strict
Session ID: 2020-03-30_20-24-26



In [6]:
!prodigy train ner cord_19_abstracts_strict models/en_core_sci_lg_no_ner --output models/2020_03_29_teach/en_rf_sci_lg

✔ Loaded model 'models/en_core_sci_lg_no_ner'
Created and merged data for 41 total examples
Using 21 train / 20 eval (split 50%)
Component: ner | Batch size: compounding | Dropout: 0.2 | Iterations: 10
ℹ Baseline accuracy: 0.000

=========================== ✨  Training the model ===========================

#    Loss       Precision   Recall     F-Score 
--   --------   ---------   --------   --------
 1    1737.49       0.000      0.000      0.000                                 
 2      25.99       0.000      0.000      0.000                                 
 3     115.63       0.000      0.000      0.000                                 
 4      24.86       0.000      0.000      0.000                                 
 5      22.36       0.000      0.000      0.000                                 
 6      18.33       0.000      0.000      0.000                                 
 7      20.52       0.000      0.000      0.000                                 
 8      28.45       0.000   